In [ ]:
import pandas as pd
import numpy as np
import csv
from scipy.stats import rankdata
from matplotlib import pyplot as plt

In [ ]:
seed_list = range(20)
m_imputations = 10
s = 50
t= 250

test_size=0.5
dataset_list = ['avila', 'biodeg', 'landcover',  'letter', 'magic',  'optdigits', 'page-blocks',  'penbased', 'ring', 'satimage', 'segment', 'spambase', 'steel', 'texture', 'twonorm', 'vehicle', 'vowel', 'waveform', 'winequality-red', 'winequality-white']
missing_rate_list = [10,20,30,40,50]
classifier_list = ['rf','nn','lr']
method_list = ['random', 'lc', 'lm', 'entropy', 'gini', 'cumir', 'proposed_lm']

In [ ]:
for dataset in dataset_list:
    for missing_rate in missing_rate_list:
        for classifier in classifier_list:
            for method in method_list:      
                    array = np.empty((0,1))
                    for seed in seed_list:
                        a = pd.read_csv('./result/{0}_{1}_{2}_{3}_{4}.csv'.format(method ,dataset, missing_rate, classifier, seed), delimiter=',', header=None)
                        a_array = np.array(a)
                        array = np.concatenate((array.reshape(t+1,-1), a_array), axis=1)
                    avg_array = np.mean(array, axis=1)
                        
                    b = open('./result/avg/{0}_{1}_{2}_{3}.csv'.format(method, dataset, missing_rate, classifier), 'w', newline='')
                    wrb = csv.writer(b)
                    for i in avg_array:
                        wrb.writerow([i])
                    b.close()

In [ ]:
for missing_rate in missing_rate_list:
    for classifier in classifier_list:
        for method in method_list:
            array = np.empty((0,1))
            for dataset in dataset_list:
                a = pd.read_csv('./result/avg/{0}_{1}_{2}_{3}.csv'.format(method, dataset, missing_rate, classifier), delimiter=',', header=None)
                a_array = np.array(a)
                start_point = a_array[0]
                a_array -= start_point
                array = np.concatenate((array.reshape(t+1,-1), a_array), axis=1)
            avg_array = np.mean(array, axis=1)
                    
            b = open('./result/avg/mai/{0}_{1}_{2}.csv'.format(method,missing_rate,classifier), 'w', newline='')
            wrb = csv.writer(b)
            for i in avg_array:
                wrb.writerow([i])
            b.close()
           

In [ ]:
iteration = 250
for classifier in classifier_list:
    b = open('./result/avg_rank/avg/iter{0}_{1}.csv'.format(iteration, classifier),'w',newline='')
    c = open('./result/avg_rank/std/iter{0}_{1}.csv'.format(iteration, classifier), 'w', newline='')
    for missing_rate in missing_rate_list:
        rank_matrix = np.zeros((20,7))
        
        for dataset_idx, dataset in enumerate(dataset_list):
            
            proposed_lm = pd.read_csv('./result/avg/proposed_lm_{0}_{1}_{2}.csv'.format(dataset, missing_rate, classifier), delimiter=',',header=None)
            proposed_lm = np.array(proposed_lm)[iteration]
            lc = pd.read_csv('./result/avg/lc_{0}_{1}_{2}.csv'.format(dataset, missing_rate, classifier), delimiter=',',header=None)
            lc = np.array(lc)[iteration]
            lm = pd.read_csv('./result/avg/lm_{0}_{1}_{2}.csv'.format(dataset, missing_rate, classifier), delimiter=',',header=None)
            lm = np.array(lm)[iteration]
            entropy = pd.read_csv('./result/avg/entropy_{0}_{1}_{2}.csv'.format(dataset, missing_rate, classifier), delimiter=',',header=None)
            entropy = np.array(entropy)[iteration]
            gini = pd.read_csv('./result/avg/gini_{0}_{1}_{2}.csv'.format(dataset, missing_rate, classifier), delimiter=',',header=None)
            gini = np.array(gini)[iteration]
            cumir = pd.read_csv('./result/avg/cumir_{0}_{1}_{2}.csv'.format(dataset, missing_rate, classifier), delimiter=',',header=None)
            cumir = np.array(cumir)[iteration]
            random = pd.read_csv('./result/avg/random_{0}_{1}_{2}.csv'.format(dataset, missing_rate, classifier), delimiter=',',header=None)
            random = np.array(random)[iteration]

            comparison_matrix = np.concatenate((random, lc, lm, entropy, gini,cumir, proposed_lm), axis=0)
            comparison_matrix = -comparison_matrix
            
            comparison_rank = rankdata(comparison_matrix)
            rank_matrix[dataset_idx] = comparison_rank
        
        average_rank_matrix = np.mean(rank_matrix, axis=0)
        average_rank_matrix = np.round(average_rank_matrix,2)
        
        std_rank_matrix = np.std(rank_matrix, axis=0)
        std_rank_matrix = np.round(std_rank_matrix,2)
        print('iter:',iteration, 'clf:', classifier,'missing:', missing_rate)
        
        wrb = csv.writer(b)
        wrb.writerow(average_rank_matrix)
        wrc = csv.writer(c)
        wrc.writerow(std_rank_matrix)
    b.close()
    c.close()

In [ ]:
for classifier in classifier_list:
    avg = pd.read_csv('./result/avg_rank/avg/iter{0}_{1}.csv'.format(iteration, classifier), delimiter=',', header=None).values
    std = pd.read_csv('./result/avg_rank/std/iter{0}_{1}.csv'.format(iteration, classifier), delimiter=',', header=None).values
    
    d = open('./result/avg_rank/total/iter{0}_{1}.csv'.format(iteration, classifier), 'w', newline='')
    for i in range(avg.shape[0]):
        
        aList=[]
        for j in range(avg.shape[1]):
            avg_val = avg[i,j]
            std_val = std[i,j]
            total_str = '%.2f±%.2f'%(avg_val, std_val)
            aList.append(total_str)
        print(aList)
        wrd = csv.writer(d)
        wrd.writerow(aList)
    d.close()